In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('coordinate of New York City are {}, {}.'.format(latitude, longitude))

coordinate of New York City are 40.7127281, -74.0060152.


In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [7]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [8]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [9]:
CLIENT_ID = 'WJDFBOKTGOMFCFCTXBEJR2DHAWHAB5RRZSLOTIGCYUCVG0NV'
CLIENT_SECRET = 'PMQRPQDQQKCNUA1H21JFRIH5KNN3QY45IZEAJGLKB3OLELDX'
VERSION = '20210615' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500 # define radius

In [10]:
#https://developer.foursquare.com/docs/resources/categories
#Asian = 4bf58dd8d48988d142941735

neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_asian = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d142941735')
newyork_venues_asian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,China Wang,40.874347,-73.910540,Chinese Restaurant
1,Marble Hill,40.876551,-73.91066,Mei Chung Mei Restaurant,40.878272,-73.902981,Chinese Restaurant
2,Marble Hill,40.876551,-73.91066,Anise,40.885989,-73.910036,Chinese Restaurant
3,Marble Hill,40.876551,-73.91066,Kam Wah Kitchen,40.876084,-73.908698,Asian Restaurant
4,Marble Hill,40.876551,-73.91066,Moon Star Grill,40.886238,-73.909961,Asian Restaurant


In [11]:
newyork_venues_asian.shape

(1927, 7)

In [12]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [17]:
map_newyork_asian = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_asian, 'red', map_newyork_asian)

map_newyork_asian

In [18]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [19]:
manhattan_grouped = newyork_venues_asian.groupby('Neighborhood').count()
manhattan_grouped
# print('There are {} uniques categories.'.format(len(newyork_venues_asian['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,50,50,50,50,50,50
Carnegie Hill,50,50,50,50,50,50
Central Harlem,50,50,50,50,50,50
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
East Harlem,49,49,49,49,49,49
East Village,50,50,50,50,50,50


In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_asian[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_asian['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Beer Bar,Burger Joint,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Cocktail Bar,Coffee Shop,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Gastropub,Grocery Store,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Hunan Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Karaoke Bar,Korean BBQ Restaurant,Korean Restaurant,Malay Restaurant,Mexican Restaurant,New American Restaurant,Noodle House,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Thai Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Xinjiang Restaurant
0,Marble Hill,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Bakery,Beer Bar,Burger Joint,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Cocktail Bar,Coffee Shop,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Gastropub,Grocery Store,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Hunan Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Karaoke Bar,Korean BBQ Restaurant,Korean Restaurant,Malay Restaurant,Mexican Restaurant,New American Restaurant,Noodle House,Peking Duck Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Thai Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Xinjiang Restaurant
0,Battery Park City,0.040000,0.00,0.00,0.020000,0.000000,0.00,0.00,0.160000,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.00,0.00,0.00,0.00,0.040000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.180000,0.00,0.00,0.080000,0.00,0.020000,0.00,0.020000,0.00,0.00,0.060000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.200000,0.00,0.00,0.00,0.00,0.080000,0.00,0.00,0.00,0.00,0.040000,0.00,0.020000
1,Carnegie Hill,0.040000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.140000,0.000000,0.00,0.00,0.000000,0.02,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.02,0.00,0.00,0.000000,0.100000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.020000,0.00,0.00,0.040000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.280000,0.04,0.00,0.00,0.00,0.220000,0.00,0.00,0.00,0.02,0.060000,0.00,0.000000
2,Central Harlem,0.060000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.560000,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.100000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.040000,0.00,0.00,0.040000,0.00,0.00,0.00,0.00,0.020000,0.00,0.00,0.00,0.060000,0.00,0.00,0.00,0.00,0.060000,0.00,0.00,0.00,0.00,0.000000,0.02,0.000000
3,Chelsea,0.080000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.100000,0.020000,0.00,0.02,0.000000,0.02,0.020000,0.02,0.02,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.160000,0.00,0.00,0.080000,0.02,0.000000,0.00,0.060000,0.00,0.00,0.100000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.140000,0.00,0.00,0.00,0.00,0.100000,0.00,0.00,0.02,0.00,0.020000,0.00,0.000000
4,Chinatown,0.060000,0.04,0.00,0.000000,0.020000,0.02,0.02,0.180000,0.000000,0.00,0.00,0.000000,0.08,0.020000,0.02,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.04,0.00,0.00,0.00,0.02,0.000000,0.040000,0.00,0.00,0.040000,0.04,0.000000,0.00,0.020000,0.00,0.00,0.020000,0.00,0.00,0.00,0.02,0.000000,0.00,0.04,0.00,0.080000,0.00,0.00,0.02,0.00,0.060000,0.00,0.00,0.00,0.02,0.080000,0.00,0.000000
5,Civic Center,0.080000,0.04,0.00,0.000000,0.000000,0.02,0.06,0.200000,0.000000,0.00,0.00,0.000000,0.06,0.020000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.06,0.00,0.00,0.00,0.02,0.000000,0.060000,0.02,0.00,0.040000,0.02,0.000000,0.00,0.060000,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.04,0.00,0.020000,0.00,0.00,0.02,0.00,0.040000,0.00,0.00,0.00,0.02,0.080000,0.00,0.000000
6,Clinton,0.020000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.100000,0.000000,0.00,0.00,0.000000,0.02,0.000000,0.04,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.140000,0.00,0.00,0.040000,0.00,0.000000,0.00,0.020000,0.00,0.00,0.200000,0.02,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.140000,0.00,0.00,0.00,0.00,0.220000,0.00,0.00,0.00,0.00,0.020000,0.00,0.000000
7,East Harlem,0.040816,0.00,0.00,0.000000,0.000000,0.00,0.00,0.632653,0.000000,0

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Thai Restaurant,Korean Restaurant
1,Carnegie Hill,Sushi Restaurant,Thai Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant
2,Central Harlem,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant
3,Chelsea,Japanese Restaurant,Sushi Restaurant,Thai Restaurant,Chinese Restaurant,Ramen Restaurant
4,Chinatown,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Asian Restaurant


In [24]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 1, 0, 3, 3, 0, 1, 0, 0], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Chinese Restaurant,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Korean Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Asian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Sushi Restaurant,Ramen Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,1,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Dumpling Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Cocktail Bar


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,Upper West Side,Sushi Restaurant,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Asian Restaurant
13,Lincoln Square,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Vietnamese Restaurant
14,Clinton,Thai Restaurant,Ramen Restaurant,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant
17,Chelsea,Japanese Restaurant,Sushi Restaurant,Thai Restaurant,Chinese Restaurant,Ramen Restaurant
19,East Village,Japanese Restaurant,Korean Restaurant,Ramen Restaurant,Sushi Restaurant,Vietnamese Restaurant
24,West Village,Sushi Restaurant,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Noodle House
28,Battery Park City,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Thai Restaurant,Korean Restaurant
29,Financial District,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Thai Restaurant,Food Truck
34,Sutton Place,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant,Asian Restaurant,Ramen Restaurant
35,Turtle Bay,Japanese Restaurant,Chinese Restaurant,Thai Restaurant,Ramen Restaurant,Sushi Restaurant


In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Marble Hill,Chinese Restaurant,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Korean Restaurant
2,Washington Heights,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Sushi Restaurant,Ramen Restaurant
3,Inwood,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Dumpling Restaurant
4,Hamilton Heights,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Cocktail Bar
5,Manhattanville,Chinese Restaurant,Japanese Restaurant,Sushi Restaurant,Thai Restaurant,Japanese Curry Restaurant
6,Central Harlem,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant
7,East Harlem,Chinese Restaurant,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Hawaiian Restaurant


In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,Midtown,Korean Restaurant,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Thai Restaurant
16,Murray Hill,Korean Restaurant,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Ramen Restaurant
27,Gramercy,Korean Restaurant,Japanese Restaurant,Ramen Restaurant,Chinese Restaurant,Asian Restaurant
33,Midtown South,Korean Restaurant,Japanese Restaurant,Sushi Restaurant,Asian Restaurant,Ramen Restaurant
36,Tudor City,Korean Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Ramen Restaurant
38,Flatiron,Korean Restaurant,Japanese Restaurant,Sushi Restaurant,Thai Restaurant,Chinese Restaurant


In [30]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Chinatown,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Asian Restaurant
18,Greenwich Village,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Vietnamese Restaurant,Chinese Restaurant
20,Lower East Side,Korean Restaurant,Chinese Restaurant,Vietnamese Restaurant,Japanese Restaurant,Sushi Restaurant
21,Tribeca,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Japanese Restaurant,Asian Restaurant
22,Little Italy,Chinese Restaurant,Sushi Restaurant,Asian Restaurant,Dim Sum Restaurant,Thai Restaurant
23,Soho,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Vietnamese Restaurant
25,Manhattan Valley,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Noodle House
26,Morningside Heights,Chinese Restaurant,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Korean Restaurant
31,Noho,Sushi Restaurant,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,Thai Restaurant
32,Civic Center,Chinese Restaurant,Asian Restaurant,Vietnamese Restaurant,Noodle House,Cha Chaan Teng


In [31]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Upper East Side,Sushi Restaurant,Thai Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant
9,Yorkville,Sushi Restaurant,Chinese Restaurant,Thai Restaurant,Japanese Restaurant,Ramen Restaurant
10,Lenox Hill,Sushi Restaurant,Thai Restaurant,Japanese Restaurant,Chinese Restaurant,Asian Restaurant
11,Roosevelt Island,Sushi Restaurant,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Asian Restaurant
30,Carnegie Hill,Sushi Restaurant,Thai Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant


In [ ]:
newyork_venues_asian